In [1]:
# Load the required libraries
from openai import OpenAI
import random
import subprocess
import pathlib
from dotenv import load_dotenv
import os
import xmlrpc.client
import time
import re
import base64

In [2]:
# Load the API key from the .env file
load_dotenv()

api_key = os.getenv("API_KEY")

client = OpenAI(api_key=api_key)

In [3]:
sys_instruction="All prompts should be answered with an in depth paper with an introduction, middle and end structured into chapters that is about 4 pages, written in markdown and include sources. Dont answer anything with less than 4 pages. Dont write anything other than the paper. Dont write ```markdown etc."
topics = ["technology", "science", "history", "art", "literature", "politics", "economics", "philosophy"]

plagiarism_score = 0
version = 1
file_path = f""
topic = ""

for paper_number in range(1, 30):
    topic = random.choice(topics)
    completion = client.chat.completions.create(
    model="gpt-4o",
    store=True,
    messages=[
        {"role": "developer", "content": sys_instruction},
        {"role": "user", "content": f"Write a paper about {topic}. The specific topic is up to you."}
    ]
    )


    try: 
        # Write the response to a file
        with open(f"outputs/{paper_number}_{topic}_version{version}.md", "a") as f:
            f.write(completion.choices[0].message.content)
        file_path = f"outputs/{paper_number}_{topic}_version{version}"
        print('Written to file')
    
    except Exception as e:
        print(f"Error writing to file: {e}")

    # Convert the markdown file to a PDF
    command = f'pandoc "{file_path}.md" --pdf-engine=xelatex -o "{file_path}.pdf'
    subprocess.run(command, shell=True, check=True)

Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file
Written to file


In [19]:
# Authenticate with iThenticate
load_dotenv()

username = os.getenv("ITHENTICATE_USERNAME")
password = os.getenv("ITHENTICATE_PASSWORD")

url = "https://api.ithenticate.com/rpc"
server = xmlrpc.client.ServerProxy(url)

credentials = {
    'username': username,
    'password': password
}

response = server.login(credentials)
sid = response['sid']
sid_dict =  dict(sid = response['sid'])
print(response['api_status'])

200


In [20]:
# Find the folder ID for 'Trym Master Thesis' folder
response = server.folder.list(sid_dict)

folder_id = None
for folder in response.get('folders', []):
    if folder.get('name') == 'Trym Master Thesis':
        folder_id = folder.get('id')
        break

folder = dict(folder = folder_id)
print(f"Folder ID for 'Trym Master Thesis': {folder}")

Folder ID for 'Trym Master Thesis': {'folder': 4378261}


In [6]:
# Define the folder containing the PDFs
folder_path = 'outputs'

# Initialize the array to hold document data
documents = []

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        # Extract the title from the filename (assuming the title is the filename without extension)
        title = os.path.splitext(filename)[0]
        
        author_first = 'OpenAI'
        author_last = 'ChatGPT'
        
        # Read the PDF file and encode its content in base64
        with open(os.path.join(folder_path, filename), 'rb') as pdf_file:
            encoded_pdf = xmlrpc.client.Binary(pdf_file.read())
        
        # Create the document data dictionary
        document_data = {
            'title': title,
            'author_first': author_first,
            'author_last': author_last,
            'filename': filename,
            'upload': encoded_pdf
        }
        
        # Add the document data to the array
        documents.append(document_data)

# Update the dictionary with the documents array
arguments = dict(sid=sid, folder=folder_id, submit_to=1, uploads=documents)

# Submit the documents to iThenticate
response = server.document.add(arguments)
print(response['api_status'])

200


In [21]:
arguments = dict(sid=sid, id=folder_id)

response = server.folder.get(arguments)
# Extract document IDs and store them in a list
document_ids = [doc['id'] for doc in response['documents'] if doc.get('author_last') == "ChatGPT"]

print(document_ids)

[115388832, 115388827, 115388840, 115388821, 115388841, 115388835, 115388816, 115388842, 115388837, 115388830, 115388824, 115388834, 115388833, 115388823, 115388829, 115388839, 115388822, 115388828, 115388820, 115388817, 115388836, 115388838, 115388825, 115388818, 115388814]


In [40]:
# Iterate through the document IDs
for document_id in document_ids:
    arguments = dict(sid=sid, id=document_id)
    response = server.document.get(arguments)
    plagiarism_score = response['documents'][0]['parts'][0]['score']
    document_title = response['documents'][0]['title']
    
    # Extract the paper number and topic from the document title
    match = re.match(r'^\D*(\d{1,2})', document_title)
    paper_number = match.group(1) if match else ''
    
    match = re.search(r'_(.*?)_', document_title)
    topic = match.group(1) if match else ''
    
    paper_version = int(document_title[-1])

    # Check plagiarism score and generate a new paper if the score is above 10% a maximum of 10 times
    for paper_version in range(1, 10):
        #Check plagiarism score
        if plagiarism_score >= 10:
            print(f"{document_title} has plagiarism score: {plagiarism_score}%")
        
            # Path to the PDF file
            doc_path = f"outputs/{document_title}.pdf"
            
            # Read the PDF file bytes
            with open(doc_path, 'rb') as doc:
                data = doc.read()
            base64_string = base64.b64encode(data).decode("utf-8")

            # Generate a new paper based on the previous one
            prompt = f"You got got cought plagiarizing. The plagirism score was {plagiarism_score}%. Write the paper again to fix the plagiarism."
            sys_instruction="All prompts should be answered with an in depth paper with an introduction, middle and end structured into chapters that is about 4 pages, written in markdown and include sources. Dont answer anything with less than 4 pages. Dont write anything other than the paper. The paper should be about the same topic as the previous one. With as few changes as possible."

            completion = client.chat.completions.create(
                model="gpt-4o",
                store=True,
                messages=[
                    {"role": "developer", "content": sys_instruction},
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "file",
                                "file": {
                                    "filename": f"{document_title}.pdf",
                                    "file_data": f"data:application/pdf;base64,{base64_string}",
                                },
                            },
                            {
                                "type": "text",
                                "text": f"You got got cought plagiarizing. The plagirism score was {plagiarism_score}%. Write the paper again to fix the plagiarism.",
                            },
                        ],
                    },
                ],
            )

            try: 
                # Write the response to a file
                file_path = f"outputs/{paper_number}_{topic}_version{paper_version+1}"
                with open(f"outputs/{paper_number}_{topic}_version{paper_version+1}.md", "a") as f:
                    f.write(completion.choices[0].message.content)

                command = [
                            "pandoc",
                            f"{file_path}.md",
                            "--pdf-engine=xelatex",
                            "-o",
                            f"{file_path}.pdf"
                        ]

                subprocess.run(command, check=True)
                print(f"written to {file_path} to file")
                
                # Initialize the array to hold document data
                documents = []
            
                title = f"{paper_number}_{topic}_version{paper_version+1}"
                
                author_first = 'OpenAI'
                author_last = 'ChatGPT'
                
                # Read the PDF file and encode its content in base64
                with open(f"outputs/{title}.pdf", 'rb') as pdf_file:
                    encoded_pdf = xmlrpc.client.Binary(pdf_file.read())
                

                # Create the document data dictionary
                document_data = {
                    'title': title,
                    'author_first': author_first,
                    'author_last': author_last,
                    'filename': filename,
                    'upload': encoded_pdf
                }

                # Add the document data to the array
                documents.append(document_data)

                # Update the test dictionary with the documents array
                arguments = dict(sid=sid, folder=folder_id, submit_to=1, uploads=documents)

                # Submit the documents to iThenticate
                response = server.document.add(arguments)
                document_id = response['uploaded'][0]['id']
                print('Document uploaded to iThenticate')
                time.sleep(20)
                
                arguments = dict(sid=sid, id=document_id)
                response = server.document.get(arguments)

                plagiarism_score = response['documents'][0]['parts'][0]['score']
                document_title = response['documents'][0]['title']
            except Exception as e:
                print(f"Error writing to file: {e}")
                pass
        else:
            break

27_science_version1 has plagiarism score: 24%
written to outputs/27_science_version2 to file
Document uploaded to iThenticate
27_science_version2 has plagiarism score: 27%
written to outputs/27_science_version3 to file
Document uploaded to iThenticate
27_science_version3 has plagiarism score: 16%
written to outputs/27_science_version4 to file
Document uploaded to iThenticate
27_science_version4 has plagiarism score: 15%
written to outputs/27_science_version5 to file
Document uploaded to iThenticate
27_science_version5 has plagiarism score: 16%
written to outputs/27_science_version6 to file
Document uploaded to iThenticate
27_science_version6 has plagiarism score: 19%
written to outputs/27_science_version7 to file
Document uploaded to iThenticate
27_science_version7 has plagiarism score: 18%
written to outputs/27_science_version8 to file
Document uploaded to iThenticate
27_science_version8 has plagiarism score: 25%
written to outputs/27_science_version9 to file
Document uploaded to iThe